In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta
import os 
from supabase import create_client, Client
from dotenv import load_dotenv


In [2]:
#cargar el DATASET
customer_df = pd.read_csv(r"C:\Users\jorge\OneDrive\Escritorio\uf\customer.csv", sep=';')
team_df = pd.read_csv(r"C:\Users\jorge\OneDrive\Escritorio\uf\team.csv", sep=';')
condominium_df = pd.read_csv(r"C:\Users\jorge\OneDrive\Escritorio\uf\condominum.csv", sep=',')

### Se usa .head para verificar las columnas y la informacion dentro de los dataframe

In [3]:
customer_df.head()

,id_customer,dni,first_name,last_name,email,phone_number,country,province,address,postal_code,participation_quantity,prospect_status,isEnable,client_status,createdAt,updateAt,team_id
0,1,49-896-8694,Ula,Blades,ublades0@disqus.com,7488907019,Argentina,Mendoza,9382 Raven Circle,NaN,NaN,Closed,False,Active,2/10/2024,10/07/2024,5
1,2,28-040-9556,Tammie,Cocke,tcocke1@macromedia.com,6430893677,Argentina,Catamarca,188 Calypso Drive,4750-419,NaN,Contacted,False,Inactive,1/10/2024,10/07/2024,4
2,3,85-078-8085,Palmer,Luscombe,pluscombe2@phpbb.com,5842608251,Argentina,Chaco,1153 Maple Wood Road,50130,NaN,Contacted,False,Inactive,1/10/2024,10/09/2024,5
3,4,07-184-2391,Kirk,Baudins,kbaudins3@mapy.cz,7994836755,Argentina,Chubut,8 Bunting Alley,NaN,NaN,Contacted,True,Inactive,1/10/2024,10/09/2024,3
4,5,73-849-0899,Leshia,Armin,larmin4@google.ca,5653062669,Argentina,Catamarca,51 Clemons Place,70760,NaN,Contacted,True,Inactive,2/10/2024,10/08/2024,2


In [4]:
team_df.head()

,team_id,first_name,last_name,email,phone_number,country,created_at
0,1,Vendedor,1,paskham0@sciencedaily.com,1093378816,Argentina,15/09/2024
1,2,Vendedor,2,pmcmechan1@noaa.gov,1117621162,Argentina,15/09/2024
2,3,Vendedor,3,ctanser2@japanpost.jp,8701151460,Argentina,6/09/2024
3,4,Vendedor,4,ffleischmann3@wikia.com,1895167965,Argentina,28/09/2024
4,5,Vendedor,5,cheartfield4@dmoz.org,7141032673,Argentina,19/09/2024


In [5]:
condominium_df.head()

,id_condominum,condominum_name,propieties,condominum_status,createdAt,updateAt,id_customer
0,1,Carter Group,258,Active,18/10/2024,04/11/2024,23
1,2,Marvin Group,152,Active,08/10/2024,03/11/2024,25
2,3,Braun-Hartmann,226,Active,02/10/2024,03/11/2024,6
3,4,Hills and Sons,263,Inactive,06/10/2024,01/11/2024,21
4,5,Raynor LLC,300,Inactive,13/10/2024,03/11/2024,33


### Se realiza limpieza de los datos, normalizando datos de DNI a solo numero, dandole formato de fecha a las columnas con fechas, quitando columnas innecesarias para los calculos

In [6]:
#Funcion para limpiar los caractereres incorrectos en la casilla de dni, phone_number y postal_code
def convert_to_number(x):
    if pd.isna(x):
        # Si el valor es NaN, devolver NaN
        return x
    elif isinstance(x, str):
        # Filtrar solo los dígitos de la cadena
        filtered_str = ''.join(filter(str.isdigit, x))
        if filtered_str:
            return int(filtered_str)  # Convertir directamente a entero
        else:
            return None  # Si no quedan dígitos después de la limpieza, devolver None
    elif isinstance(x, float):
        # Convertir un número float a int si es necesario
        return int(x)
    else:
        # Si ya es un número entero, devolverlo tal cual
        return x


In [7]:
customer_df['dni'] = customer_df['dni'].apply(convert_to_number)
customer_df['phone_number'] = customer_df['phone_number'].apply(convert_to_number)
customer_df['postal_code'] =customer_df['postal_code'].apply(convert_to_number)

In [8]:
# Convertir la columna 'createdAt' a formato de fecha
customer_df['createdAt'] = pd.to_datetime(customer_df['createdAt'])
# Establecer 'createdAt' como el índice
customer_df.set_index('createdAt', inplace=True)
# Formato para mostrar las fechas como DD/MM/YY
customer_df.index = customer_df.index.strftime('%d/%m/%y')
customer_df.sort_values('createdAt', inplace=True)

In [9]:
# Se eliminan las columnas no necesarias para los calculos de la tabla customers
customer_df.drop(['postal_code', 'participation_quantity', 'email', 'phone_number', 'address'], axis=1, inplace=True)
customer_df.head()

,id_customer,dni,first_name,last_name,country,province,prospect_status,isEnable,client_status,updateAt,team_id
createdAt,,,,,,,,,,,
10/01/24,38,29260942,Alia,Cadman,Argentina,Rio Negro,Proposed,True,Inactive,10/07/2024,4
10/01/24,26,854640512,Modesty,Rraundl,Argentina,Formosa,Closed,True,Active,10/08/2024,3
10/01/24,20,58408494,Stevena,Pleaden,Argentina,Salta,Negociation,False,Inactive,10/08/2024,2
10/01/24,18,772039989,Moll,Smallsman,Argentina,Rio Negro,Contacted,True,Inactive,10/09/2024,3
10/01/24,45,726962816,Bunnie,Spadotto,Argentina,La Rioja,Contacted,False,Inactive,10/09/2024,3


# Calculo de totales necesarios

#### Para ejecutar todas las funciones se usa el dataset de customer llamado customer_df

## Totales sin filtro por mes 

In [10]:
# Total clientes en estado propuesta
def total_customer_proposed(df):
    #Realización de cálculos
    Total_Clientes_Propuesta = customer_df[customer_df['prospect_status'] == 'Proposed'].shape[0]
    return Total_Clientes_Propuesta

In [11]:
# Total clientes en estado negociacion
def total_customer_negociation(df):
    #Realización de cálculos
    Total_Clientes_Negociacion = customer_df[customer_df['prospect_status'] == 'Negociation'].shape[0]
    return  Total_Clientes_Negociacion

In [12]:
# Total clientes en estado contactado
def total_customer_contated(df):
    #Realización de cálculos
    Total_Clientes_Contactado = customer_df[customer_df['prospect_status'] == 'Contacted'].shape[0]
    return  Total_Clientes_Contactado
    

In [13]:
# Total Clientes en estado Cerrado
def total_customer_closed(df):
    #Realización de cálculos
    Total_Clientes_Cerrados = customer_df[customer_df['prospect_status'] == 'Closed'].shape[0]
    return Total_Clientes_Cerrados,


In [14]:
# Total facturacion por cliente 
def total_customer_facturation (df):
    #Realizacion del calculo
    join = pd.merge(customer_df,condominium_df,on='id_customer',how='inner')
    total_propiedades = join.groupby('id_customer')['propieties'].sum().reset_index(name='total_propiedades')
    total_propiedades['total_propiedades'] = total_propiedades['total_propiedades'].apply(lambda x: f"${x:,.2f}")
    return total_propiedades

In [15]:
# Total de condominios por cada cliente
def total_customer_condominum (df):
    #realizacion del calculo
    join = pd.merge(customer_df,condominium_df,on='id_customer', how='inner')
    total_condominios = join.groupby('id_customer')['propieties'].count().reset_index(name='total_condominios')
    return total_condominios

In [16]:
# Total clientes activos
def total_customer_active(df):
    Total_Clientes_Activos = customer_df[customer_df['isEnable']==True].shape[0]
    return Total_Clientes_Activos

In [17]:
# Total clientes inactivos 
def total_customer_inactive(df):
    Total_Clientes_Inactivos = customer_df[customer_df['isEnable']==False].shape[0]
    return Total_Clientes_Inactivos

In [18]:
# Total clientes en prospectos en total sin importar los meses
def total_prospect(df):
    total_prospectos = customer_df[customer_df['prospect_status'] != 'Closed'].shape[0]
    return total_prospectos

## Totales Filtrados por Mes

### Todos estos filtros se ejecutan en base a la fecha de createdAt, en caso de actualizacion de estado, sigue tomando la fecha de creacion, no toma en cuenta la fecha de actualizacion

In [19]:
#Funcion para filtrar por mes en la visualizacion, tomada por numeros del 1 al 12 
def monthly_filter(df, mes):
    # Extraemos el mes del índice (que es la columna 'createdAt')
    df_filtrado = customer_df[customer_df.index.to_series().apply(lambda x: pd.to_datetime(x).month) == mes]
    return df_filtrado

In [20]:
# Total clientes totales prospectos en todos los estados, prospecto, contactado, negociacion y cerrado en el mes
def total_customer_prospect_month(df, mes):
    df_filtrado = monthly_filter(df,mes)
    #prospectos para el mes
    total_clientes_propuesta_mes = df_filtrado.shape[0]
    
    return total_clientes_propuesta_mes

In [21]:
# Total clientes cerrados por mes 
def total_customer_closed_month(df, mes):
    df_filtrado = monthly_filter(df,mes)
    total_cliente_cerrado_mes = df_filtrado[df_filtrado['prospect_status']== 'Closed'].shape[0]
    return total_cliente_cerrado_mes


In [22]:
# Total clientes en estado contactado por mes
def total_customer_contacted_month (df, mes):
    df_filtrado = monthly_filter(df,mes)
    total_cliente_contactado_mes = df_filtrado[df_filtrado['prospect_status'] == 'Contacted'].shape[0]
    return total_cliente_contactado_mes

In [23]:
# Total clientes en estado negociacion por mes 
def total_customer_negociation_month (df,mes):
    df_filtrado = monthly_filter(df,mes)
    total_cliente_negociacion_mes = df_filtrado[df_filtrado['prospect_status']== 'Negociation'].shape[0]
    return total_cliente_negociacion_mes

In [24]:
# Total clientes en estado propuesta por mes
def total_customer_proposed_month(df,mes):
    df_filtrado = monthly_filter(df,mes)
    Total_Clientes_Propuesta_mes = df_filtrado[df_filtrado['prospect_status'] == 'Proposed'].shape[0]
    return Total_Clientes_Propuesta_mes

In [25]:
# Total clientes activos por mes
def total_customer_active_month(df,mes):
    df_filtrado = monthly_filter(df,mes)
    Total_Clientes_Activos_mes = df_filtrado[df_filtrado['isEnable']==True].shape[0]
    return Total_Clientes_Activos_mes

In [26]:
# Total de clientes inactivos por mes
def total_customer_inactive_month(df,mes):
    df_filtrado = monthly_filter(df,mes)
    Total_Clientes_Inactivos_mes = df_filtrado[df_filtrado['isEnable']==False].shape[0]
    return Total_Clientes_Inactivos_mes

## Metas mensuales

In [27]:
# Porcentaje de meta mensual
def monthly_target_porcent (df,mes):
    total = total_customer_prospect_month(df,mes)
    meta_mensual_porcentaje = total * 0.75
    porcentaje = meta_mensual_porcentaje/total_customer_prospect_month(df,mes) * 100
    return porcentaje

In [28]:
# Meta Mensual
def monthly_target(df,mes):
    total = total_customer_prospect_month(df,mes)
    meta_mensual = total * 0.75
    return meta_mensual

In [29]:
# Porcentaje de cumplimiento al momento 
def compliance_rate(df,mes):
    total_prospectos = total_customer_prospect_month(df,mes)
    meta = total_prospectos * 0.75
    if meta > 0:
        cumplimiento = (total_customer_closed_month(df,mes)/meta) *100
    else:
        cumplimiento = 0 #Evitar division por 0

    return cumplimiento

In [30]:
print(f"Total Prospectos Mensual:",total_customer_prospect_month(customer_df,10))
print(f"Meta Mensual: {monthly_target(customer_df,10):.1f}%")
print(f"Clientes Cerrados:",total_customer_active_month(customer_df,10))
print(f"Cumplimiento Mes: {compliance_rate(customer_df,10):.1f}%")


Total Prospectos Mensual: 50
Meta Mensual: 37.5%
Clientes Cerrados: 26
Cumplimiento Mes: 26.7%
